<a href="https://colab.research.google.com/github/betty6you/Jing_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**



(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#CovidVaccine"](https://twitter.com/hashtag/CovidVaccine) from Twitter. 


In [ ]:
# Write your code here
# I will collect the all the customer reviews of the product 2019 Dell labtop on Amazon
#!pip install Scrapy

#!scrapy startproject AmazonReviewScraping
#!scrapy genspider DellReviews https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=

import scrapy


class AmazonReviewsSpider(scrapy.Spider):

  name = "reviews"
  allowed_domains=['amazon.com']
  myBaseUrl="https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/product-reviews/B07N49F51N/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber="
  startURL=[]
  for i in range (1,13):
    startURL.append(myBaseUrl+str(i))
    
  def parse(self, response):
    #Get the Review List
    data = response.css('#cm_cr-review_list')
    
    #Get the Name
    name = data.css('.a-profile-name')
            
    #Get the Review Title
    title = data.css('.review-title')
            
    # Get the Ratings
    star_rating = data.css('.review-rating')
    # Get the users Comments
    comments = data.css('.review-text')
    count = 0
           
    # combining the results
    for review in star_rating:
      yield{
      'Rating': ''.join(review.xpath('.//text()').extract()),
      'Comment': ''.join(comments[count].xpath(".//text()").extract())
         }
      count=count+1

# run file in terminal !scrapy runspider AmazonReviewScraping/AmazonReviewScraping/spiders/DellSpider.py -o reviews.csv    
      

2021-02-23 22:51:14 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-02-23 22:51:14 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.10 (default, Feb 20 2021, 21:17:23) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2021-02-23 22:51:14 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-02-23 22:51:15 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2021-02-23 22:51:15 [scrapy.extensions.telnet] INFO: Telnet Password: 7874a178458d9af3
2021-02-23 22:51:15 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-02-23 22:51

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [14]:
# Write your code here

!pip install pandas
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')

data=pd.read_csv(r"https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/Review.csv")
data['comment'] = data['comment'].apply(lambda x: x.replace(r"'", ''))
data['comment'] = data['comment'].apply(lambda x: x.replace(r'"', ''))
data['comment'] = data['comment'].apply(lambda x: x.replace(r'[\n\n\n\n\n\n\n\n\n\n  \n  \n    , \n  ', ''))
data['comment'] = data['comment'].apply(lambda x: x.replace(r'\n, \n  \n]', ''))

data['stopwords']=data['comment'].apply(lambda x:len ([x for x in x.split( ) if x in stop])) #remove stopwords
data['special_char']=data['comment'].apply(lambda x: len([x for x in x.split() if x.startswith(('**','Â','â€','©','€™'))])) #Remove sepcial characters as noise

data['commnet'] = data['comment'].str.rstrip(string.digits)#remove numbers
data['comment']=data['comment'].str.replace('[^\w\s]','') # punctuation removal

data['comment']=data['comment'].apply(lambda x:" ".join(x.lower() for x in x.split()))# lower casing

from nltk.stem import PorterStemmer
st = PorterStemmer()
data['comment'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()])) # Stemming

from textblob import Word
nltk.download('wordnet')
data['comment']= data['comment'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) # Lemmatization

data['comment'].to_csv("AmazonReviewupdated.csv",header=None,index=False) # only 1 column with clean sentences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Question 3

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Write your code here






**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [ ]:
'''
Write your explanations of the constituency parsing tree and dependency parsing tree here



'''

'\nWrite your explanations of the constituency parsing tree and dependency parsing tree here\n\n\n\n'